In [22]:
import json
from sklearn.externals import joblib
import os
import re
import numpy as np

In [5]:
dataPath = '../data'

docUrlsName = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'
samplePath = '../data/sample.csv'

queriesDocsPath = '../data/queries.docs.txt'
queriesPath = '../data/queries.numerate.txt'

In [13]:
procNumbs = set(map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath)))

queriesDict = {}
with open(queriesPath) as inputFile:
    for line in inputFile:
        number, query = line.strip().split('\t')
        queriesDict[number] = query
        
queriesDocsDict = json.load(open(queriesDocsPath))

In [40]:
class TfIdfFeatures:
    trainTfIdfPath = '../data/trainDataTfIdf'
    tfIdfVectorizerPath = '../data/models/tfIdfVectorizer.pkl'
    tfIdfNumberPagesPath = '../data/tfIdfNumberPages'
    tfIdfPagesPath = '../data/tfidfPages'
    tfIdfFeaturesPath = '../data/tfidfFeatutes'
    
    def __init__(self):
        numberPages = json.load(open(TfIdfFeatures.tfIdfNumberPagesPath))
        features = json.load(open(TfIdfFeatures.tfIdfFeaturesPath))
        
        self._indexPages = np.full(27000, -1, dtype=np.int)
        self._indexPages[numberPages] = range(len(numberPages))
        self._pages = joblib.load(TfIdfFeatures.tfIdfPagesPath)        
        self._indexFeatures = dict(zip(features, range(features)))
        
    def tfidf(self, word, numberDoc):
        pass
        
    

In [41]:
tfidfFeatures = TfIdfFeatures()

TypeError: 'list' object cannot be interpreted as an integer

In [31]:
numberPages[119]

126

In [32]:
index[126]

119